### RAG Evaluator
We have created a RAG system that can answer questions based on the provided context. There is also a evlaution set that is created by an LLM synthetically.
The eval set contains the query, the answer, and the relevant context. The next step is to evaluate the RAG system using the eval set on the following criteria:
- Relevancy of the answer to the query
- Correctness of the answer
- Relevancy of the answer to the context
- Correctness of the citation
- Safety of the answer

The evaluation is done in the following steps:
1. The RAG system is used to answer the query
2. The answer is compared to the answer in the eval set
3. The citation is compared to the citation in the eval set


In [2]:
from pydantic import BaseModel, Field
from typing import Dict, List, Optional, Any
from pathlib import Path
import json
import os
import instructor
from openai import OpenAI
from prompts import system_prompt_rag_eval_bot

c:\Users\shres\anaconda3\envs\rag_case_study\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class EvaluationMetrics(BaseModel):
    relevancy_score: float = Field(
        ge=0, le=1, 
        description="Score indicating how relevant the response is to the query (0-1). Compare the is_relevant field to the question and the context provided and"
         "decide whether field is_relevant is correct or not."
    )
    correctness_score: float = Field(
        ge=0, le=1, 
        description="Score indicating factual correctness of the llm_response compared to ground_truth_answer (0-1)"
    )
    context_alignment_score: float = Field(
        ge=0, le=1, 
        description="Score indicating how well the llm_response aligns with  retrieved context (0-1). The score is 0 if the llm_response"
        "is completely irrelevant to the retrieved chunks and 1 if it perfectly alligns with the context provided in retrieved chunks"
    )
    citation_score: float = Field(
        ge=0, le=1,
    description="Score indicating the accuracy of cited information compared to the retrieved context. Must check if the information picked"
    "out from a particular cited chunk is answered in the retrieved chunk.")
    safety_score: float = Field(
        ge=0, le=1, 
        description="Score indicating safety of the response (0-1)"
    )
    feedback: str = Field(
        description="Detailed feedback explaining the scores"
    )



In [11]:
class RAGEvaluator:
    def __init__(
        self,
        openai_api_key: str,
        model: str = "o3-mini",
        max_retries: int = 3
    ):
        self.client = instructor.patch(OpenAI(api_key=openai_api_key))
        self.model = model
        self.max_retries = max_retries
        
    def load_document_chunks(self, experiment_number: str) -> Dict[str, Dict]:
        """Load document chunks from JSON file"""
        chunks_file = os.path.join("Experiments", experiment_number, "document_chunks.json")
        with open(chunks_file, 'r') as f:
            chunks_data = json.load(f)
        # Create a lookup dictionary with chunk_id as key
        return {chunk["chunk_id"]: chunk for chunk in chunks_data}

    def format_retrieved_chunks(self, chunk_ids: List[str], chunks_lookup: Dict[str, Dict]) -> str:
        """Format retrieved chunks into a single string"""
        formatted_chunks = []
        for chunk_id in chunk_ids:
            if chunk_id in chunks_lookup:
                chunk = chunks_lookup[chunk_id]
                formatted_chunk = (
                    f"Chunk ID: {chunk_id}\n"
                    f"Document: {chunk['document_name']}\n"
                    f"Content: {chunk['chunk_content']}\n"
                )
                formatted_chunks.append(formatted_chunk)
        return "\n".join(formatted_chunks)

    def format_cited_chunks(self, cited_chunks: Optional[Dict[str, str]]) -> str:
        """Format cited chunks into a single string"""
        if cited_chunks is None:
            return "No citations provided"
            
        formatted_citations = []
        for chunk_id, content in cited_chunks.items():
            formatted_citation = (
                f"Chunk ID: {chunk_id}\n"
                f"Content: {content}\n"
            )
            formatted_citations.append(formatted_citation)
        return "\n".join(formatted_citations)

    def evaluate_response(self, record: Dict[str, Any], chunks_lookup: Dict[str, Dict]) -> EvaluationMetrics:
        """Evaluate a single response record"""
        
        # Format retrieved chunks
        retrieved_chunks_str = self.format_retrieved_chunks(
            record['retrieved_chunk_ids'], 
            chunks_lookup
        )
        # print(retrieved_chunks_str)
        
        # Format cited chunks
        cited_chunks_str = self.format_cited_chunks(record['cited_chunk_ids'])
        # print(cited_chunks_str)
        
        evaluation_prompt = system_prompt_rag_eval_bot.format(
            question=record['question'],
            ground_truth_answer=record['ground_truth_answer'],
            llm_response=record['llm_response'],
            is_relevant=record['is_relevant'],
            retrieved_chunks = retrieved_chunks_str,
            cited_chunks=cited_chunks_str
        )

        try:
            evaluation = self.client.chat.completions.create(
                model=self.model,
                response_model=EvaluationMetrics,
                messages=[
                    {"role": "system", "content": "You are an expert evaluator of RAG systems."},
                    {"role": "user", "content": evaluation_prompt}
                ],
                max_retries=self.max_retries
            )
            return evaluation
        except Exception as e:
            print(f"Error evaluating question '{record['question']}': {str(e)}")
            return None

    def evaluate_experiment(self, experiment_number: str) -> Dict[str, Any]:
        """Evaluate all responses in an experiment"""
        
        # Load the evaluation set
        eval_file = os.path.join("Experiments", experiment_number, "llm_responses_eval_set.json")
        with open(eval_file, 'r') as f:
            eval_set = json.load(f)

        # Load document chunks
        chunks_lookup = self.load_document_chunks(experiment_number)

        # Evaluate each record
        evaluations = []
        for record in eval_set:
            print(f"Evaluating question: {record['question']}")
            evaluation = self.evaluate_response(record, chunks_lookup)
            if evaluation:
                evaluations.append(evaluation.model_dump())

        # Calculate average scores
        avg_scores = {
            "avg_relevancy_score": sum(e["relevancy_score"] for e in evaluations) / len(evaluations),
            "avg_correctness_score": sum(e["correctness_score"] for e in evaluations) / len(evaluations),
            "avg_context_alignment_score": sum(e["context_alignment_score"] for e in evaluations) / len(evaluations),
            "avg_citation_score": sum(e["citation_score"] for e in evaluations) / len(evaluations),
            "avg_safety_score": sum(e["safety_score"] for e in evaluations) / len(evaluations),
        }

        # Prepare final output
        final_output = {
            "experiment_number": experiment_number,
            "individual_evaluations": evaluations,
            "average_scores": avg_scores
        }

        # Save results
        output_file = os.path.join("Experiments", experiment_number, "evaluation_results.json")
        with open(output_file, 'w') as f:
            json.dump(final_output, f, indent=2)

        return final_output

In [12]:
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
evaluator = RAGEvaluator(
    openai_api_key=OPENAI_API_KEY,
    model="o3-mini"
)

results = evaluator.evaluate_experiment("001")

Evaluating question: What is the AI Act's objective?
Evaluating question: When will AI Act be applicable?
Evaluating question: What does "AI System" mean?
Evaluating question: What does "putting into service" mean regarding an AI system?
Evaluating question: Who is considered a "provider"?
Evaluating question: What is "informed consent" in testing?
Evaluating question: What is a "deep fake"?
Evaluating question: What's the definition of "widespread infringement"?
Evaluating question: What does "critical infrastructure" mean?
Evaluating question: Who is responsible for ensuring AI literacy?
Evaluating question: CCPA effective date?
Evaluating question: Which entities must comply with CCPA?
Evaluating question: Who is considered a 'Consumer' under CCPA?
Evaluating question: What constitutes 'Personal Information' under CCPA?
Evaluating question: What is 'selling' Personal Information under CCPA?
Evaluating question: What does GDPR stand for?
Evaluating question: When was GDPR adopted?
Ev